<a href="https://colab.research.google.com/github/Cipe96/EEG-Recognition/blob/main/Classificatore_Convolutivo.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

<font size=6>**EEG Recognition: Classificatore Convolutivo**</font>
</br><font size=3>*Marco Cipollina, Riccardo Era*</font>


<p style="font-size:4px;" align="justify">Dopo aver svolto vari tipo di analisi sul nostro dataset e viste le varie possibilità di preprocessing attuabile in "Analisi Dataset e Preprocessing", in questo notebook, viene costruito e addestrato un modello Convolutivo (CCN) semplice. Questo classificatore sarà utilizzato come baseline per ricercare la migliore combinazione di preprocessing da sfruttare succesivamente in un modello più complesso.</p>
<p style="font-size:4px;" align="justify">Qui sotto è perciò possibile caricare e lavorare su un determinato dataset e salvarne i risultati. I dati da noi ottenuti saranno confrontati ed esposti nel noteobok "Risultati".</p>

<font size=4>**Indice:**</font>
*   [Import librerie](#1)
*   [Downloads](#2)
*   [Classificatore](#3)

<a name="1"></a>
# **Import librerie**

Importiamo le librerie e montiamo Google Drive per garantire l'accesso agli altri file.

In [ ]:
from sklearn.utils import shuffle
import matplotlib.pyplot as plt
from google.colab import drive
import tensorflow as tf
import pandas as pd
import numpy as np
import sys
import os

SEED = 96 #Impostiamo un seme specifico per garantire la replicabilità degli esperimenti

In [ ]:
%%capture
drive.mount('/content/drive', force_remount=True)
# elimina la cartella sample_data creata automaticamente
! rm -r /content/sample_data

<a name="2"></a>
# **Download**

In [ ]:
#@title Seleziona la banda che vuoi utilizzare:
#@markdown (delta, theta, alpha, beta, gamma, broadband, personalizzata, ABG)

banda = 'ABG' #@param ['delta', 'theta', 'alpha', 'beta', 'gamma', 'broadband', 'personalizzata', 'ABG']
save_path = '/content/drive/MyDrive/EEG Recognition/Dati preprocessati/'+banda+'/'

In [ ]:
# Carica i dati e le etichette
train_data = np.load(f"{save_path}/train_data_{banda}.npy")
train_labels = np.load(f"{save_path}/train_labels_{banda}.npy")
val_data = np.load(f"{save_path}/val_data_{banda}.npy")
val_labels = np.load(f"{save_path}/val_labels_{banda}.npy")
test_data = np.load(f"{save_path}/test_data_{banda}.npy")
test_labels = np.load(f"{save_path}/test_labels_{banda}.npy")

Controlliamo che la shape sia coerente con quella che ci servirà per il modello di classificazione

In [ ]:
# Verifica le dimensioni dei dataset e delle etichette
print(f"Train set shape: {train_data.shape}, Train labels shape: {train_labels.shape}")
print(f"Validation set shape: {val_data.shape}, Validation labels shape: {val_labels.shape}")
print(f"Test set shape: {test_data.shape}, Test labels shape: {test_labels.shape}")

Train set shape: (3052, 240, 64), Train labels shape: (3052,)
Validation set shape: (654, 240, 64), Validation labels shape: (654,)
Test set shape: (654, 240, 64), Test labels shape: (654,)


Effettuiamo uno shuffle del dataset per evitare di dare in pasto i dati al modello in maniera ordinata, permettendo così un addestramento meno condizionato.

In [ ]:
#shuffle dei dataset e ettichette con stesso ordine di shuffle
train_data, train_labels = shuffle(train_data, train_labels, random_state=SEED)
val_data, val_labels = shuffle(val_data, val_labels, random_state=SEED)
test_data, test_labels = shuffle(test_data, test_labels, random_state=SEED)


<a name="3"></a>
# **Classificatore**

In [ ]:
#Riprendiamo il numero di classi esistenti
num_classes = len(np.unique(train_labels))
print("Numero di classi:", num_classes)

# Convertiamo le etichette in one-hot encoding
y_train = tf.keras.utils.to_categorical(train_labels, num_classes=num_classes)
y_val = tf.keras.utils.to_categorical(val_labels, num_classes=num_classes)
y_test = tf.keras.utils.to_categorical(test_labels, num_classes=num_classes)


Numero di classi: 109


Costruiamo il nostro modello convoluzionale utilizzando alcune delle best practices (SeparableConv, DropOut, BatchNormalization) e uno strato Dense per permetterci di classificare i nostri volontari.

In [ ]:
#Impostiamo i parametri per i blocchi convolutivi
filtri = 16
dropout = 0.6

# Aggiungi un layer di input all'inizio per specificare la forma
inputs = tf.keras.layers.Input(shape=(train_data.shape[1], train_data.shape[2]))

# Utilizziamo strati convoluzionali per estrarre caratteristiche locali
for i in range(3):
  if i == 0:
    x = tf.keras.layers.Conv1D(filters=filtri, kernel_size=3, padding='same', activation='relu')(inputs)
  else:
    x = tf.keras.layers.SeparableConv1D(filters=filtri, kernel_size=3, padding='same', activation='relu')(inputs)

  x = tf.keras.layers.BatchNormalization()(x)
  x = tf.keras.layers.Dropout(dropout)(x)

  filtri *= 2
  dropout /= 2

#Appiattiamo tutto primo di consegnare al Dense
x = tf.keras.layers.GlobalAveragePooling1D()(x)

# Aggiungiamo uno strato Dense per la classificazione finale
outputs = tf.keras.layers.Dense(num_classes, activation='softmax')(x)

model = tf.keras.Model(inputs=inputs, outputs=outputs)

# Compiliamo il modello
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

Verifichiamo che il modello sia costruito come vogliamo.

In [ ]:
model.summary()

Model: "functional_28"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                         ┃ Output Shape                ┃         Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━┩
│ input_layer_28 (InputLayer)          │ (None, 240, 64)             │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ separable_conv1d_57                  │ (None, 240, 64)             │           4,352 │
│ (SeparableConv1D)                    │                             │                 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ batch_normalization_86               │ (None, 240, 64)             │             256 │
│ (BatchNormalization)                 │                             │                 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dropout_86 (Dropout)                 │ (None, 240, 64)             │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ global_average_pooling1d_28          │ (None, 64)                  │               0 │
│ (GlobalAveragePooling1D)             │                             │                 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_28 (Dense)                     │ (None, 109)                 │           7,085 │
└──────────────────────────────────────┴─────────────────────────────┴─────────────────┘

 Total params: 11,693 (45.68 KB)

 Trainable params: 11,565 (45.18 KB)

 Non-trainable params: 128 (512.00 B)

Creiamo una callback per salvare i pesi con i quali si otterranno i risultati migliori sul validation set e avviamo il training.

In [ ]:
#Salviamo i pesi dell'epoca in cui abbiamo ottenuto il val_loss minore
callback = [
    tf.keras.callbacks.ModelCheckpoint(filepath="Convolutivo.keras", save_best_only=True, monitor="val_loss"),
]

# Addestriamo il modello
history = model.fit(train_data, y_train, epochs=50, batch_size=32, validation_data=(val_data, y_val), callbacks=callback)


Epoch 1/50
96/96 ━━━━━━━━━━━━━━━━━━━━ 8s 51ms/step - accuracy: 0.0395 - loss: 4.5907 - val_accuracy: 0.0856 - val_loss: 4.6048
Epoch 2/50
96/96 ━━━━━━━━━━━━━━━━━━━━ 1s 5ms/step - accuracy: 0.1588 - loss: 4.1363 - val_accuracy: 0.2477 - val_loss: 4.3915
Epoch 3/50
96/96 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step - accuracy: 0.2792 - loss: 3.7599 - val_accuracy: 0.3654 - val_loss: 3.9901
Epoch 4/50
96/96 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step - accuracy: 0.3696 - loss: 3.3871 - val_accuracy: 0.4862 - val_loss: 3.4436
Epoch 5/50


Per ricaricare i pesi che hanno performato meglio durante l'addestramento:

In [ ]:
model = tf.keras.models.load_model("Convolutivo.keras")

Valutiamo infine le prestazioni del modello sul test set.

In [ ]:
loss, accuracy = model.evaluate(test_data, y_test, verbose=0)
print(f'Accuratezza sul test: {accuracy:.4f}')

Accuratezza sul test: 0.9893
